# Problem Set I

Nessa lista de exercícios testaremos comandos básicos para carregar e manipular um dataset em Python. O dataset que usaremos ao longo do curso são os dados do ENEM de 2017. Os dados estão disponíveis gratuitamente no site do INEP:

http://download.inep.gov.br/microdados/microdados_enem2017.zip

O primeiro passo é baixar os dados. No site do INEP você poderá encontrar vários outros datasets, incluindo microdados do ENEM desde 2004. 


## **Atividade 1**: Baixe os dados do ENEM, e salve na pasta `dados`. *Cuidado:* os dados são grandes ($\approx$1GB).

Agora vamos manipular os dados usando um pacote chamado `pandas`. Uma introdução ao `pandas` pode ser encontrada aqui:

* https://towardsdatascience.com/pandas-dataframe-a-lightweight-intro-680e3a212b96
* https://pandas.pydata.org/pandas-docs/stable/getting_started/10min.html

In [1]:
import pandas as pd  # biblioteca para manipular dados em dataframes
import numpy as np   # biblioteca para manipulação numérica em python
import matplotlib.pyplot as plt # biblioteca para gerar gráficos em python.

O póximo comando carrega os dados. Ajuste a variável `filepath` para ser a pasta dos dados no seu computador. Os dados serão armazenados em um objeto em `pandas` chamado o `dataframe`, denotado aqui por `df`. 

Os dados são carregados usando o comando `pd.read_csv`. Note que vamos carregar apenas as 500k primeiras linhas dos dados (i.e., dados de apenas 500k estudantes). Delete o parâmetro `nrows` em `pd.read_csv` para carregar os dados inteiros. Note que isso pode demorar muito dependendo do seu computador.

In [2]:
# Load data -- esse comando vai demorar um pouco por causa do tamanho dos dados.
filepath = '../data/Microdados ENEM 2017/DADOS/MICRODADOS_ENEM_2017.csv' # data path -- CHANGE to match where your data is saved.
df = pd.read_csv(filepath,encoding='cp860',sep=';', nrows=500000)        # load first 250k rows of the data and fix encoding
# df = pd.read_csv(filepath,encoding='cp860',sep=';', nrows=250000)      # load the entire dataset.

df.head()  # print the first five lines

,NU_INSCRICAO,NU_ANO,CO_MUNICIPIO_RESIDENCIA,NO_MUNICIPIO_RESIDENCIA,CO_UF_RESIDENCIA,SG_UF_RESIDENCIA,NU_IDADE,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,...,Q018,Q019,Q020,Q021,Q022,Q023,Q024,Q025,Q026,Q027
0,170003336736,2017,3503208,Araraquara,35,SP,29.0,F,0.0,1,...,A,C,B,B,C,B,B,B,A,A
1,170003333545,2017,5002902,CassilΓndia,50,MS,22.0,F,0.0,1,...,A,B,A,A,C,B,A,A,A,A
2,170001663644,2017,3550308,Sπo Paulo,35,SP,38.0,F,0.0,1,...,A,B,A,A,C,A,B,B,A,A
3,170001663645,2017,4209300,Lages,42,SC,35.0,F,0.0,1,...,B,C,A,B,D,A,B,B,A,A
4,170001663646,2017,2704302,Macei≤,27,AL,40.0,M,0.0,3,...,A,B,B,A,C,A,C,B,A,A


Vamos ver quais são os atributos do nosso dataset. Uma explicação dos atributos pode ser encontrada na pasta `Dicionário` onde os dados form baixados.

In [3]:
print('Size of dataset: ' + str(df.shape ))
print(list(df))  # print feature list

Size of dataset: (500000, 137)
['NU_INSCRICAO', 'NU_ANO', 'CO_MUNICIPIO_RESIDENCIA', 'NO_MUNICIPIO_RESIDENCIA', 'CO_UF_RESIDENCIA', 'SG_UF_RESIDENCIA', 'NU_IDADE', 'TP_SEXO', 'TP_ESTADO_CIVIL', 'TP_COR_RACA', 'TP_NACIONALIDADE', 'CO_MUNICIPIO_NASCIMENTO', 'NO_MUNICIPIO_NASCIMENTO', 'CO_UF_NASCIMENTO', 'SG_UF_NASCIMENTO', 'TP_ST_CONCLUSAO', 'TP_ANO_CONCLUIU', 'TP_ESCOLA', 'TP_ENSINO', 'IN_TREINEIRO', 'CO_ESCOLA', 'CO_MUNICIPIO_ESC', 'NO_MUNICIPIO_ESC', 'CO_UF_ESC', 'SG_UF_ESC', 'TP_DEPENDENCIA_ADM_ESC', 'TP_LOCALIZACAO_ESC', 'TP_SIT_FUNC_ESC', 'IN_BAIXA_VISAO', 'IN_CEGUEIRA', 'IN_SURDEZ', 'IN_DEFICIENCIA_AUDITIVA', 'IN_SURDO_CEGUEIRA', 'IN_DEFICIENCIA_FISICA', 'IN_DEFICIENCIA_MENTAL', 'IN_DEFICIT_ATENCAO', 'IN_DISLEXIA', 'IN_DISCALCULIA', 'IN_AUTISMO', 'IN_VISAO_MONOCULAR', 'IN_OUTRA_DEF', 'IN_GESTANTE', 'IN_LACTANTE', 'IN_IDOSO', 'IN_ESTUDA_CLASSE_HOSPITALAR', 'IN_SEM_RECURSO', 'IN_BRAILLE', 'IN_AMPLIADA_24', 'IN_AMPLIADA_18', 'IN_LEDOR', 'IN_ACESSO', 'IN_TRANSCRICAO', 'IN_LIBRAS', 'IN

O dataset do enem tem vários atributos (uma descrição pode ser encontrada na pasta onde os dados foram baixados). Os próximos comandos reduzem o dataset retirando, por exemplo, os treineiros e indivíduos que faltaram à prova ou foram eliminados/desqualificados.

In [4]:
# Remove all entries that were absent or were eliminated in at least one exam
ix = ~df[['TP_PRESENCA_CN','TP_PRESENCA_CH','TP_PRESENCA_LC','TP_PRESENCA_MT']].applymap(lambda x: False if x==1.0 else True).any(axis=1)
df = df.loc[ix,:]


# Remove "treineiros" -- these are individuals that marked that they are taking the exam "only to test their knowledge". 
# It is not uncommon for students to take the ENEM in the middle of high school as a dry run
df = df.loc[df['IN_TREINEIRO']==0,:]

# drop eliminated features
df.drop(['TP_PRESENCA_CN','TP_PRESENCA_CH','TP_PRESENCA_LC','TP_PRESENCA_MT','IN_TREINEIRO'],axis=1, inplace=True)

# subsitute race by names
features = ['N/A', 'Branca','Preta','Parda','Amarela','Indigena']
df['TP_COR_RACA'] = df.loc[:,['TP_COR_RACA']].applymap(lambda x: features[x]).copy()

# print new shape
print('New shape of the dataset:' + str(df.shape))

# reset index -- this commands resets the index of the dataset
df.reset_index(drop=True, inplace=True)

New shape of the dataset:(350162, 132)


Note que o dataset foi reduzido para menos de 300k entradas. Entradas individuais do dataset podem ser acessados através do comando `df.loc[]`. Por exemplo, vamos imprimir a nota de matemática do aluno na décima linha do dataset.

In [5]:
df.loc[10,'NU_NOTA_MT']

795.0

Um dataframe com apenas uma coluna em `pandas` se chama um `Series`. Vamos imprimir uma série composta apenas das notas de matemática.

In [6]:
df['NU_NOTA_MT']

0         465.5
1         591.2
2         584.6
3         578.5
4         607.5
5         479.0
6         656.5
7         430.1
8         645.1
9         517.6
10        795.0
11        425.5
12        430.8
13        481.7
14        769.1
15        790.4
16        535.5
17        622.8
18        472.9
19        672.2
20        696.1
21        459.2
22        422.4
23        539.7
24        439.9
25        799.6
26        545.0
27        405.1
28        873.2
29        493.9
          ...  
350132    577.1
350133    399.0
350134    576.8
350135    630.2
350136    415.6
350137    377.7
350138    390.5
350139    567.3
350140    730.5
350141    721.4
350142    402.6
350143    470.6
350144    617.7
350145    624.5
350146    642.2
350147    579.9
350148    586.3
350149    400.4
350150    580.3
350151    714.8
350152    501.0
350153    618.3
350154    532.4
350155    524.5
350156    375.3
350157    571.5
350158    466.5
350159    783.2
350160    432.5
350161    546.5
Name: NU_NOTA_MT, Length

Nós podemos facilmente calcular a nota média, mediana, e variância das notas de matemática utilizando funções já implementadas dentro do objeto df.

In [9]:
avg = df['NU_NOTA_MT'].mean() # mean math score
med = df['NU_NOTA_MT'].median()  # median math score
std = df['NU_NOTA_MT'].std()  # std var of the math score

print('Nota média: ' + str(avg))
print('Nota mediana: ' + str(med))
print('Desvio padrão: ' + str(std))

Nota média: 534.7401696928665
Nota mediana: 521.2
Desvio padrão: 108.43701795923563


## **ATIVIDADE 2:** Calcule as notás médias, mínimas, e máximas em ciências humanas, ciências naturais, e línguas.

In [ ]:
# entre sua resposta aqui




Agora vamos testar alumas manipulações adicionais no dataset. Para podermos analisar alunos que estão na mesma etapa educacional, vamos manter apenas os candidatos que estão concluindo o ensino médio em 2017. Esses candidatos estão (supostamente) no último ano do ensino médio. Essa etapa de pré-processamento reduz significamente o número de alunos que permanecerão no dataset.

In [11]:
df = df.loc[df.TP_ST_CONCLUSAO == 2]

# remember to reset the index!
df.reset_index(inplace=True, drop=True)

# print new shape
print('New shape of the dataset:' + str(df.shape))

New shape of the dataset:(131479, 132)


Uma outra forma de acessar atributos individuais é usar o comando `df.NOME_DO_ATRIBUTO` onde, obviamente, `NOME_DO_ATRIBUTO` é o nome do atributo que você quer acessar. Para ilustar o seu uso, vamos calcular a quantidade de estudantes de diferentes raças. 

**Observação:** Estamos utilizando a categorização de raças encontrada nos dados do ENEM. Reconhecemos que essas categorias podem ser consideradas ofensivas e são ultrapassadas. Porém, para preservar a consistência dos dados, manteremos a nomenclatura originalmente encontrada nos dados.

In [12]:
print('Percentage of candidates from each ethnicity:')
print('---------------------')
print(100*df.TP_COR_RACA.value_counts()/len(df))

print('\nTotal number of candidates from each ethnicity (in thousands):')
print('---------------------')
print(df.TP_COR_RACA.value_counts()/1000)

Percentage of candidates from each ethnicity:
---------------------
Branca      43.556005
Parda       40.856715
Preta       10.847360
Amarela      2.334213
N/A          1.940234
Indigena     0.465474
Name: TP_COR_RACA, dtype: float64

Total number of candidates from each ethnicity (in thousands):
---------------------
Branca      57.267
Parda       53.718
Preta       14.262
Amarela      3.069
N/A          2.551
Indigena     0.612
Name: TP_COR_RACA, dtype: float64


## **Atividade 3:** Calcule o número e percentagem de estudantes por estado.

In [13]:
# Complete o exercício aqui.



Agora vamos agrupar o dataset por estado (UF) e calcular a nota média em ciências naturais por estado. Isso é feito através do comando `groupby`, o que consideramos uma das funções mais poderosas do pacote `pandas`. Você pode ler mais sobre esse comando aqui:  

* https://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html

In [27]:
mean_series = df.groupby(['SG_UF_RESIDENCIA'])['NU_NOTA_CH'].mean()  # calcular a média por estado
mean_series.sort_values(inplace=True,ascending=False)                # ordenar a série
print(mean_series)

SG_UF_RESIDENCIA
MG    559.101746
SC    550.631041
PR    549.789271
RJ    549.746506
ES    549.717240
DF    549.094902
SP    546.404691
RS    545.423227
RN    543.308323
PB    541.855160
PI    541.648318
SE    538.366271
GO    537.033347
PA    534.092729
BA    533.209822
AP    529.580077
RR    528.093385
MS    527.106997
PE    525.665273
MT    524.904547
MA    522.053589
TO    521.141789
AC    519.450314
AL    519.362816
RO    518.073101
CE    516.251490
AM    504.365155
Name: NU_NOTA_CH, dtype: float64


## **ATIVIDADE 4:** Imprima o nome e estado dos munícipios de onde vieram as dez maiores notas de matemática no nosso dataset.

In [28]:
# Código aqui


## **ATIVIDADE 5:** Encontre a nota média em ciências humanas agrupando por raça e gênero.

Note que você vai precisar agrupar o dataset por dois atributos ao invés de um -- leia a documentação/guias para aprender como fazer isso.

In [29]:
# Código aqui
